### Re-do your "BDP Assignment 3" using Jupyter Notebooks and Spark RDD


You have zip file loaded into /project2/msca/kadochnikov/data/Municipal_Court_Caseload_Information.zip in Linux environment, representing the caseload for Austin municipal courts.

In [1]:
# ensuring I am using the right kernel
print (sc.version)

3.0.1


In [2]:
# imports
from itertools import islice

### 1. Copy the file into your own directory on Linux server

In [3]:
# checking the file is present at the specified location
!ls -l /project2/msca/kadochnikov/data/Municipal_Court_Caseload_Information.zip

-rwxr-xr-x 1 kadochnikov kadochnikov 94212181 Apr 16  2015 /project2/msca/kadochnikov/data/Municipal_Court_Caseload_Information.zip


In [4]:
# copying the file into my own directory
!cp /project2/msca/kadochnikov/data/Municipal_Court_Caseload_Information.zip /project2/msca/bharadwajk

In [5]:
# checking if copied
!ls -l /project2/msca/bharadwajk/Municipal_Court_Caseload_Information.zip

-rwxr-xr-x 1 bharadwajk msca 94212181 Feb 11 23:29 /project2/msca/bharadwajk/Municipal_Court_Caseload_Information.zip


### 2. Unzip the file


In [11]:
# unzipping the file
!unzip /project2/msca/bharadwajk/Municipal_Court_Caseload_Information.zip -d /project2/msca/bharadwajk/ 

Archive:  /project2/msca/bharadwajk/Municipal_Court_Caseload_Information.zip
  inflating: /project2/msca/bharadwajk/Municipal_Court_Caseload_Information.csv  


In [8]:
# checking if unzipped file present in my linux directory
!ls -l /project2/msca/bharadwajk/Municipal_Court_Caseload_Information.csv

-rw-rw-r-- 1 bharadwajk msca 877080187 Apr 16  2015 /project2/msca/bharadwajk/Municipal_Court_Caseload_Information.csv


### 3. Read the data into Spark RDD

In [28]:
# reading the data into RDD
Municipal_Court_Caseload_Information_RDD = sc.textFile("file:////project2/msca/bharadwajk/Municipal_Court_Caseload_Information.csv")
print(Municipal_Court_Caseload_Information_RDD.count())

8485777


In [27]:
# checking the first 5 records
Municipal_Court_Caseload_Information_RDD.take(5)

['Offense Case Type,Offense Date,Offense Time,Offense Charge Description,Offense Street Name,Offense Cross Street Check , Offense Cross Street,School Zone,Construction Zone,Case Closed',
 'TR,04/29/2010 07:00:00 AM +0000,22.40.00,FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY,8000 BLOCK RESEARCH,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,22.40.00,FAILURE TO SIGNAL INTENT TO CHANGE LANES,8000 BLOCK RESEARCH,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,20.00.00,SPEEDING-STATE HIGHWAYS,1000 BLOCK NORTH U S 183,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,20.00.00,NO SEAT BELT-DRIVER/PASSENGER,1000 BLOCK NORTH U S 183,N, ,N,N,Y']

### 4. Ensure you process the header record correctly

In [11]:
# using the filter transformation to return a new RDD without header
MCC_Information_RDD = Municipal_Court_Caseload_Information_RDD.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
MCC_Information_RDD.count()

8485776

In [12]:
# checking the first 5 records
MCC_Information_RDD.take(5)

['TR,04/29/2010 07:00:00 AM +0000,22.40.00,FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY,8000 BLOCK RESEARCH,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,22.40.00,FAILURE TO SIGNAL INTENT TO CHANGE LANES,8000 BLOCK RESEARCH,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,20.00.00,SPEEDING-STATE HIGHWAYS,1000 BLOCK NORTH U S 183,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,20.00.00,NO SEAT BELT-DRIVER/PASSENGER,1000 BLOCK NORTH U S 183,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,23.45.00,SPEEDING - STATE HIGHWAYS - Less than 10% over limit,1000 BLOCK RESEARCH,N, ,N,N,Y']

### 5. Calculate frequency of offenses by Offense Case Type

Creating an RDD having only the Offense Case Type information

In [23]:
# RDD only having Offense Case Type information
# Offense Case Type is the first column in the dataset
MCC_Information_OCT = MCC_Information_RDD.map(lambda s: s.split(",")[0])
MCC_Information_OCT.take(10)

['TR', 'TR', 'TR', 'TR', 'TR', 'PK', 'PK', 'PK', 'PK', 'PK']

Replacing each original value in the input RDD with a 2-tuple containing the value of Offense Case Type in the first position and the integer value 1 in the second position

In [24]:
# mapping
MCC_Information_OCT_mapped = MCC_Information_OCT.map(lambda x: (x, 1))

# checking first few items in tuple
MCC_Information_OCT_mapped.take(10)

[('TR', 1),
 ('TR', 1),
 ('TR', 1),
 ('TR', 1),
 ('TR', 1),
 ('PK', 1),
 ('PK', 1),
 ('PK', 1),
 ('PK', 1),
 ('PK', 1)]

Offese case type is the key word and lambda function will sum up the word counts for each word.

The first element in the tuple is the Offense Case Type and the second element is the frequency of that particular offense.

In [16]:
# reducing by key
MCC_Information_OCT_reduced = MCC_Information_OCT_mapped.reduceByKey(lambda x,y:x+y)

# checking the results
MCC_Information_OCT_reduced.take(5)

[('TR', 4313221),
 ('PK', 3388981),
 ('CM', 319078),
 ('CO', 240308),
 ('RL', 224188)]

### 6. Identify the most frequent offenses by Offense Charge Description (Show Offense Charge Description and offense frequency count in descending order)

In [17]:
# creating RDD only having Offense Charge Description information
# Offense Case Type is the 4th column in the dataset
MCC_Information_OCD = MCC_Information_RDD.map(lambda s: s.split(",")[3])
MCC_Information_OCD.take(10)

['FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY',
 'FAILURE TO SIGNAL INTENT TO CHANGE LANES',
 'SPEEDING-STATE HIGHWAYS',
 'NO SEAT BELT-DRIVER/PASSENGER',
 'SPEEDING - STATE HIGHWAYS - Less than 10% over limit',
 'PAY STATION RECEIPT NOT DISPLAYED',
 'EXPIRED PAY STATION RECEIPT',
 'EXPIRED PAY STATION RECEIPT',
 'EXPIRED PAY STATION RECEIPT',
 'PAY STATION RECEIPT NOT DISPLAYED']

In [18]:
#mapping
MCC_Information_OCD_mapped = MCC_Information_OCD.map(lambda x: (x, 1))

# checking first few items in tuple
MCC_Information_OCD_mapped.take(10)

[('FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY', 1),
 ('FAILURE TO SIGNAL INTENT TO CHANGE LANES', 1),
 ('SPEEDING-STATE HIGHWAYS', 1),
 ('NO SEAT BELT-DRIVER/PASSENGER', 1),
 ('SPEEDING - STATE HIGHWAYS - Less than 10% over limit', 1),
 ('PAY STATION RECEIPT NOT DISPLAYED', 1),
 ('EXPIRED PAY STATION RECEIPT', 1),
 ('EXPIRED PAY STATION RECEIPT', 1),
 ('EXPIRED PAY STATION RECEIPT', 1),
 ('PAY STATION RECEIPT NOT DISPLAYED', 1)]

In [19]:
# reducing by key
MCC_Information_OCD_reduced = MCC_Information_OCD_mapped.reduceByKey(lambda x,y:x+y)

# checking the results
MCC_Information_OCD_reduced.take(5)

[('BICYCLE - RAN RED LIGHT', 2397),
 ('RAN RED LIGHT', 157783),
 ('CMV - UNSAFE CONDITION-396 3 A 1', 509),
 ('PARKING - SIDEWALK AREA', 9918),
 ('CAMPING IN A PUBLIC AREA', 12144)]

The key in the tuple is Offense Charge Description and value is the frequency of that particular offense.

For the most frequent Offense Charge Description, in descending order of frequencies. The key will be the frequency and value will be Offense Charge Description.

Also, we have to swap the tuple data first.

In [25]:
# swapping the elements
MCC_Information_OCD_swapped = MCC_Information_OCD_reduced.map(lambda x:(x[1],x[0]))

# checking the first few items in tuple
MCC_Information_OCD_swapped.take(10)

[(2397, 'BICYCLE - RAN RED LIGHT'),
 (157783, 'RAN RED LIGHT'),
 (509, 'CMV - UNSAFE CONDITION-396 3 A 1'),
 (9918, 'PARKING - SIDEWALK AREA'),
 (12144, 'CAMPING IN A PUBLIC AREA'),
 (3042, 'SEATBELT - PASSENGER'),
 (3266, 'ALCOHOL - SELLING/POSSESSING IN PROHIBITED AREA'),
 (2820, 'PARKING - DOUBLE PARKED'),
 (43, 'MOTOR VEHICLE INSPECTION VIOLATION'),
 (6866, 'CROSSING PROPERTY TO TURN RIGHT OR LEFT')]

Sorting by key


In [26]:
# sorting by key
MCC_Information_OCD_sorted = MCC_Information_OCD_swapped.sortByKey(ascending=False)

# checking the results
MCC_Information_OCD_sorted.take(10)

[(892013, 'PAY STATION RECEIPT NOT DISPLAYED'),
 (732605, 'EXPIRED PAY STATION RECEIPT'),
 (486576, 'SPEEDING-STATE HIGHWAYS'),
 (372339, 'NO DRIVERS LICENSE'),
 (345162, 'SPEEDING - POSTED CITY STREET'),
 (337672, 'FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY'),
 (310816, 'PARKING - EXPIRED METER'),
 (287570, 'SPEEDING - STATE HIGHWAY'),
 (260662, 'FAIL TO MAINTAIN FINANCIAL RESP'),
 (238168, 'TOW AWAY ZONE NO PARKING AREA')]